In [ ]:
## CELDA 1 — LIBRARIES AND PARAMETERS

suppressPackageStartupMessages({
  library(ape)
  library(phytools)
  library(readr)
  library(ggplot2)
  library(dplyr)
  library(mvMORPH)
  library(stringr)
})

options(stringsAsFactors = FALSE)

CFG <- list(
  FILE_TREE_MCC   = "../trees/BEAST_MCC.tre",
  FILE_FOREST     = "../trees/BEAST_posterior_aligned.nex",
  FILE_TRAITS_CSV = "../traits/traits_high-level_aligned.csv",
  PATH_OUT        = "../results",

  VAR_TARGET = 0.95,                 # target cumulative variance
  K_MAX      = 30,                   # max PCs
  KSET       = c(10, 15, 20, 25, 30),# k values to test

  XLIM   = c(0, 1.05),  # x-axis limits for lambda
  WIDTH  = 8,          # inches
  HEIGHT = 6,
  DPI    = 300          # high resolution
)

OUT <- list(
  PNG      = file.path(CFG$PATH_OUT, "violin_lambda_posterior_by_k.png"),
  CSV_MCC  = file.path(CFG$PATH_OUT, "lambda_MCC_by_k.csv"),
  CSV_POST = file.path(CFG$PATH_OUT, "lambda_posterior_summary_by_k.csv")
)

ensure_dir <- function(p){
  if (!dir.exists(p)) dir.create(p, recursive = TRUE, showWarnings = FALSE)
}


In [ ]:
## CELDA 2 — LECTURA + PCA + λ EN MCC

read_data <- function(cfg){
  # Árbol MCC
  stopifnot(file.exists(cfg$FILE_TREE_MCC))
  tree <- ape::read.tree(cfg$FILE_TREE_MCC)
  stopifnot(inherits(tree, "phylo"))

  # Traits YA alineados (mismo orden que el MCC)
  stopifnot(file.exists(cfg$FILE_TRAITS_CSV))
  traits <- readr::read_csv(cfg$FILE_TRAITS_CSV,
                            show_col_types = FALSE,
                            progress = FALSE) |>
    as.data.frame()

  stopifnot("species" %in% colnames(traits))
  # Chequeo simple: mismo orden que el MCC
  stopifnot(identical(tree$tip.label, traits$species))

  rownames(traits) <- traits$species
  traits$species <- NULL

  cat("tips:", length(tree$tip.label),
      "| rasgos:", paste(dim(traits), collapse = " x "), "\n")

  list(tree = tree, traits = traits)
}

pca_from_traits <- function(traits, cfg){
  X   <- as.matrix(traits)
  Xsc <- scale(X, center = TRUE, scale = TRUE)
  pca <- prcomp(Xsc, center = FALSE, scale. = FALSE)
  pve <- (pca$sdev^2) / sum(pca$sdev^2)

  k95 <- which(cumsum(pve) >= cfg$VAR_TARGET)[1]
  k   <- min(k95, cfg$K_MAX, nrow(X) - 1, ncol(X))

  cat("PCs:", length(pca$sdev),
      "| k:", k,
      "| Var(k):", round(sum(pve[1:k]) * 100, 2), "%\n")

  list(pca = pca, pve = pve, k = k)
}

extract_lambda_from_fit <- function(fit){
  txt <- paste(capture.output(summary(fit)), collapse = "\n")
  m <- stringr::str_match(txt, "lambda\\s*:\\s*([0-9eE.+-]+)")
  lam <- suppressWarnings(as.numeric(m[, 2]))
  if (length(lam) == 0 || is.na(lam)) return(NA_real_)
  lam
}

fit_lambda_mv <- function(Y, tree){
  Y <- as.matrix(Y)

  # Alinear filas de Y al orden de tip.label
  if (!is.null(rownames(Y))) {
    if (!setequal(rownames(Y), tree$tip.label)) {
      stop("Y y árbol no tienen el mismo conjunto de especies.")
    }
    Y <- Y[tree$tip.label, , drop = FALSE]
  } else {
    rownames(Y) <- tree$tip.label
  }

  fit <- mvMORPH::mvgls(Y ~ 1, tree = tree, model = "lambda", method = "LL")
  extract_lambda_from_fit(fit)
}

run_mcc <- function(cfg){
  dat    <- read_data(cfg)
  tree   <- dat$tree
  traits <- dat$traits

  pca_obj <- pca_from_traits(traits, cfg)
  pca <- pca_obj$pca
  pve <- pca_obj$pve
  k   <- pca_obj$k

  # Matriz de scores
  Y <- pca$x[, 1:k, drop = FALSE]
  rownames(Y) <- rownames(traits)

  lambda_mcc <- fit_lambda_mv(Y, tree)
  cat("lambda MCC (k =", k, "):", lambda_mcc, "\n")

  # Sensibilidad en k
  Kset_raw <- cfg$KSET
  Kset <- Kset_raw[Kset_raw <= min(ncol(pca$x), nrow(traits) - 1)]
  if (!(k %in% Kset)) Kset <- sort(unique(c(Kset, k)))

  sens_list <- vector("list", length(Kset))
  for (i in seq_along(Kset)) {
    kk <- Kset[i]
    Yk <- pca$x[, 1:kk, drop = FALSE]
    rownames(Yk) <- rownames(traits)

    lam_kk <- tryCatch(
      fit_lambda_mv(Yk, tree),
      error = function(e) NA_real_
    )

    sens_list[[i]] <- data.frame(
      k       = kk,
      var_cum = sum(pve[1:kk]),
      lambda  = lam_kk
    )
  }
  sens <- dplyr::bind_rows(sens_list)
  cat("Sensibilidad MCC (k vs λ):\n")
  print(sens)

  list(
    tree       = tree,
    traits     = traits,
    pca        = pca,
    pve        = pve,
    k          = k,
    Y          = Y,
    lambda_mcc = lambda_mcc,
    sens       = sens
  )
}

In [ ]:
## CELDA 3 — POSTERIOR MCC + CSV

run_pool <- function(cfg, mcc_obj){
  forest_path <- cfg$FILE_FOREST
  if (!file.exists(forest_path)) {
    warning("Posterior sample file not found: ", forest_path)
    return(NULL)
  }

  forest <- ape::read.nexus(forest_path)
  stopifnot(inherits(forest, "multiPhylo"))

  cat("Trees in posterior sample:", length(forest), "\n")

  traits <- mcc_obj$traits
  pca    <- mcc_obj$pca
  pve    <- mcc_obj$pve

  Kset_raw <- cfg$KSET
  Kset <- Kset_raw[Kset_raw <= min(ncol(pca$x), nrow(traits) - 1)]
  if (length(Kset) == 0L) {
    stop("KSET has no valid values given the number of species/PCs.")
  }

  k_opt <- mcc_obj$k
  if (!(k_opt %in% Kset)) Kset <- sort(unique(c(Kset, k_opt)))

  cat("KSET (posterior):", paste(Kset, collapse = ", "), "\n")

  post_by_k <- vector("list", length(Kset))
  names(post_by_k) <- as.character(Kset)

  for (kk in Kset) {
    cat(">> Processing k =", kk, "...\n")

    Yk <- pca$x[, 1:kk, drop = FALSE]
    rownames(Yk) <- rownames(traits)

    lam_vec <- numeric(length(forest))
    lam_vec[] <- NA_real_

    n_ok <- 0L

    for (i in seq_along(forest)) {
      tr <- forest[[i]]

      # Align Yk to tip order of this tree
      if (!setequal(tr$tip.label, rownames(Yk))) {
        stop("Tip labels in a posterior tree do not match traits/MCC.")
      }
      Ykk <- Yk[tr$tip.label, , drop = FALSE]

      lam <- tryCatch(
        fit_lambda_mv(Ykk, tr),
        error = function(e) NA_real_
      )
      lam_vec[i] <- lam
      if (is.finite(lam)) n_ok <- n_ok + 1L
    }

    cat("k =", kk, "| OK:", n_ok, "\n")
    post_by_k[[as.character(kk)]] <- lam_vec
  }

  # Posterior summary per k
  summary_list <- vector("list", length(Kset))
  for (j in seq_along(Kset)) {
    kk <- Kset[j]
    lam_vec <- post_by_k[[as.character(kk)]]
    ok <- is.finite(lam_vec)
    n_ok <- sum(ok)

    if (n_ok > 0L) {
      vals <- lam_vec[ok]
      m    <- mean(vals)
      s    <- stats::sd(vals)  # kept though we won't write it out
      qs   <- stats::quantile(vals, probs = c(0.025, 0.25, 0.5, 0.75, 0.975),
                              names = FALSE)
    } else {
      m <- s <- NA_real_
      qs <- rep(NA_real_, 5L)
    }

    summary_list[[j]] <- data.frame(
      k        = kk,
      var_cum  = sum(pve[1:kk]),
      n_ok     = n_ok,
      mean     = m,
      sd       = s,
      q025     = qs[1],
      q250     = qs[2],
      q500     = qs[3],
      q750     = qs[4],
      q975     = qs[5],
      ci95_low  = qs[1],
      ci95_high = qs[5]
    )
  }

  post_summary <- dplyr::bind_rows(summary_list)
  cat("Posterior summary (head):\n")
  print(head(post_summary))

  list(
    KSET         = Kset,
    post_by_k    = post_by_k,
    post_summary = post_summary
  )
}

run_all <- function(cfg = CFG){
  ensure_dir(cfg$PATH_OUT)

  cat("=== MCC analysis ===\n")
  mcc_obj <- run_mcc(cfg)

  cat("\n=== Posterior analysis ===\n")
  pool_obj <- run_pool(cfg, mcc_obj)

  cat("\n=== Writing CSVs ===\n")

  ## 1) MCC CSV: one row per k (10–30) with lambda in the MCC
  mcc_df <- mcc_obj$sens |>
    dplyr::arrange(k) |>
    dplyr::rename(
      k                 = k,
      var_explained_cum = var_cum,
      lambda_MCC        = lambda
    ) |>
    #  redondeo a 3 decimales 
    dplyr::mutate(
      dplyr::across(
        c(var_explained_cum, lambda_MCC),
        ~ round(.x, 3)
      )
    )

  readr::write_csv(mcc_df, OUT$CSV_MCC)
  cat("MCC summary saved to:", OUT$CSV_MCC, "\n")

  ## 2) Posterior CSV: mean, median, 95% CI per k
  post_df <- NULL
  if (!is.null(pool_obj) && !is.null(pool_obj$post_summary)) {
    ps <- pool_obj$post_summary

    post_df <- data.frame(
      k                 = ps$k,
      var_explained_cum = ps$var_cum,
      n_trees           = ps$n_ok,
      lambda_mean       = ps$mean,
      lambda_median     = ps$q500,
      lambda_ci95_low   = ps$ci95_low,
      lambda_ci95_high  = ps$ci95_high
    ) |>
      dplyr::arrange(k) |>
      #  redondeo a 3 decimales 
      dplyr::mutate(
        dplyr::across(
          c(var_explained_cum,
            lambda_mean,
            lambda_median,
            lambda_ci95_low,
            lambda_ci95_high),
          ~ round(.x, 3)
        )
      )

    readr::write_csv(post_df, OUT$CSV_POST)
    cat("Posterior summary saved to:", OUT$CSV_POST, "\n")
  } else {
    cat("No posterior information: posterior CSV not written.\n")
  }

  invisible(list(
    cfg      = cfg,
    mcc_obj  = mcc_obj,
    pool_obj = pool_obj,
    mcc_df   = mcc_df,
    post_df  = post_df
  ))
}

res <- run_all(CFG)


In [ ]:

## CELDA 4 — PLOT (PNG)

#  Labels 
AXIS_LABEL_X <- "Number of PCs (k) from high-level features"
AXIS_LABEL_Y <- "Pagel's \u03BB"

LEG_POST <- "Posterior sample (100 trees)"
LEG_MED  <- "Posterior median"
LEG_MCC  <- "MCC"

#  Regla única para escala Y (misma que low-level) 
y_break        <- 0.2
y_round_top    <- 0.1
lambda_min_top <- 1.1

calc_ymax <- function(x, round_to = 0.1, min_top = 1.1){
  mx <- max(x, na.rm = TRUE)
  ymax <- ceiling(mx / round_to) * round_to
  if (ymax < min_top) ymax <- min_top
  ymax
}

# Estilo
base_font    <- 16
axis_text_sz <- 14

# leyenda abajo-derecha (inside)
legend_position_inside      <- c(0.98, 0.05)
legend_justification_inside <- c(1, 0)
legend_text_size            <- 12
legend_box_linewidth        <- 0.4
legend_margin               <- margin(2, 2, 2, 2)
legend_spacing_y            <- grid::unit(0.2, "lines")
legend_key_size             <- grid::unit(0.4, "lines")

violin_width    <- 0.8
violin_alpha    <- 0.6
violin_line_col <- "grey40"
violin_line_lwd <- 0.4

point_shape      <- 21
point_border_col <- "black"
point_size_mcc   <- 3
point_size_med   <- 2.6

fill_values <- c(
  MCC  = "#1f78b4",
  POST = "grey70",
  MED  = "black"
)

make_plot <- function(cfg, mcc_obj, pool_obj){

  if (is.null(pool_obj) || is.null(pool_obj$post_by_k)) {
    stop("No posterior information available for plotting.")
  }

  post_by_k    <- pool_obj$post_by_k
  post_summary <- pool_obj$post_summary

  df_violin <- dplyr::bind_rows(
    lapply(names(post_by_k), function(kk) {
      data.frame(k = as.integer(kk), lambda = post_by_k[[kk]])
    })
  )
  df_violin <- df_violin[is.finite(df_violin$lambda), , drop = FALSE]
  df_violin$k <- factor(df_violin$k)

  sens_df <- mcc_obj$sens
  sens_df$k <- factor(as.integer(sens_df$k))

  post_summary$k <- factor(as.integer(post_summary$k))

  # y_max con regla única
  y_max <- calc_ymax(
    c(df_violin$lambda, sens_df$lambda, post_summary$q500),
    round_to = y_round_top,
    min_top  = lambda_min_top
  )

  # minor breaks: entre ticks mayores (0.1, 0.3, 0.5, ...)
  y_minor <- seq(0, y_max, by = y_break / 2)

  gg <- ggplot(df_violin, aes(x = k, y = lambda)) +
    geom_violin(
      aes(fill = LEG_POST),
      width     = violin_width,
      alpha     = violin_alpha,
      colour    = violin_line_col,
      linewidth = violin_line_lwd,
      trim      = FALSE
    ) +
    geom_point(
      data = post_summary,
      aes(x = k, y = q500, fill = LEG_MED),
      inherit.aes = FALSE,
      shape  = point_shape,
      colour = point_border_col,
      size   = point_size_med
    ) +
    geom_point(
      data = sens_df,
      aes(x = k, y = lambda, fill = LEG_MCC),
      inherit.aes = FALSE,
      shape  = point_shape,
      colour = point_border_col,
      size   = point_size_mcc
    ) +
    scale_y_continuous(
      limits = c(0, y_max),
      breaks = seq(0, y_max, by = y_break),
      minor_breaks = y_minor,
      expand = expansion(mult = c(0, 0))
    ) +
    labs(x = AXIS_LABEL_X, y = AXIS_LABEL_Y) +
    scale_fill_manual(
      name   = NULL,
      values = c(
        setNames(fill_values["POST"], LEG_POST),
        setNames(fill_values["MED"],  LEG_MED),
        setNames(fill_values["MCC"],  LEG_MCC)
      ),
      breaks = c(LEG_MCC, LEG_POST, LEG_MED),
      labels = c(LEG_MCC, LEG_POST, LEG_MED)
    ) +
    guides(
      fill = guide_legend(
        override.aes = list(
          shape  = c(point_shape, 22, point_shape),
          colour = c(point_border_col, violin_line_col, point_border_col),
          alpha  = c(1, violin_alpha, 1)
        )
      )
    ) +
    theme_bw(base_size = base_font) +
    theme(
      # grid
      panel.grid.major.x = element_line(colour = "grey85", linewidth = 0.5),
      panel.grid.major.y = element_line(colour = "grey85", linewidth = 0.5),
      panel.grid.minor.x = element_blank(),
      panel.grid.minor.y = element_line(colour = "grey92", linewidth = 0.3),

      legend.position             = "inside",
      legend.position.inside      = legend_position_inside,
      legend.justification.inside = legend_justification_inside,
      legend.title                = element_blank(),
      legend.text                 = element_text(size = legend_text_size),
      legend.background           = element_rect(
        fill      = "white",
        colour    = "black",
        linewidth = legend_box_linewidth
      ),
      legend.margin    = legend_margin,
      legend.spacing.y = legend_spacing_y,
      legend.key.size  = legend_key_size,
      legend.key       = element_rect(fill = NA, colour = NA),
      axis.text  = element_text(size = axis_text_sz),
      axis.title = element_text(size = base_font)
    )

  ggsave(
    filename = OUT$PNG,
    plot     = gg,
    width    = cfg$WIDTH,
    height   = cfg$HEIGHT,
    dpi      = cfg$DPI,
    bg       = "white"
  )

  cat("Figure saved to:", OUT$PNG, "\n")
  invisible(gg)
}

plot_obj <- make_plot(CFG, res$mcc_obj, res$pool_obj)
